### Функция

In [1]:
import warnings
import pandas as pd
import numpy as np
import json
import xgboost as xgb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import empyrical as ep
import yfinance as yf
import statsmodels.api as sm
from sklearn import preprocessing
from scipy import stats
from scipy.signal import argrelextrema
import pandas_gbq
import sys, os
from google.cloud import bigquery
from google.oauth2 import service_account
from multiprocessing.pool import ThreadPool
import datetime
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import row
from bokeh.io import output_file, output_notebook
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Slope
from bokeh.models import PolyAnnotation
from bokeh.models.widgets import Tabs, Panel
import requests
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import sys
import time
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # default='warn

In [7]:
def prepare_data_frame(bd_data, currency, sql_index, yahoo_df):
    tickers = bd_data.index.tolist()
    #     print(tickers)
    Data_for_Portfolio_TOTAL = pd.DataFrame()

    for ticker in tickers:
        try:
            # =====================================================================
            io = json.loads(bd_data.loc[ticker].loc['annuals'].replace("'", '"'))
            pd.DataFrame(io['income_statement']).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('',
                                                                                                                     0).replace(
                'N/A', 0).astype(float)
            pre_date_list = pd.Series(io['Fiscal Year'])
            pre_date_list = pre_date_list.str.split("-")
            data = pre_date_list.to_list()
            names = ["Date", "Month"]
            atata = pd.DataFrame(data[:-1], columns=names)
            date_list = atata['Date'].append(pd.Series(data[-1]), ignore_index=True)
            income_df = pd.DataFrame(io['income_statement']).set_index(date_list).replace('No Debt', 0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            balance_df = pd.DataFrame(io['balance_sheet']).set_index(date_list).replace('No Debt', 0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            cashflow_df = pd.DataFrame(io['cashflow_statement']).set_index(date_list).replace('No Debt', 0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            valuation_ratios_df = pd.DataFrame(io['valuation_ratios']).set_index(date_list).replace('No Debt',
                                                                                                    0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            valuation_and_quality_df = pd.DataFrame(io['valuation_and_quality']).set_index(date_list).drop(
                ['Restated Filing Date', 'Filing Date', 'Earnings Release Date'], axis=1).replace('', 0).replace(
                'No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('N/A', 0).astype(float)
            common_size_ratios_df = pd.DataFrame(io['common_size_ratios']).set_index(date_list).replace('No Debt', 0) \
                .replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).replace(
                'Negative Tangible Equity',
                0).astype(float)
            per_share_data_df = pd.DataFrame(io['per_share_data_array']).set_index(date_list).replace('', 0) \
                .replace('No Debt', 0).replace('-', 0).replace('N/A', 0).astype(float)
            check = 1
        except:
            print('aaaaaaaaaaaaaa')
            check = 0
            pass
        
        if check == 1:
            try:
                Data_for_Portfolio = pd.DataFrame()

                Data_for_Portfolio['PE'] = valuation_ratios_df["PE Ratio"]
                Data_for_Portfolio['PB'] = valuation_ratios_df["PB Ratio"]
                Data_for_Portfolio['PS'] = valuation_ratios_df["PS Ratio"]
                Data_for_Portfolio['PEG'] = valuation_ratios_df["PEG Ratio"]
                Data_for_Portfolio['E/P'] = income_df['Net Income'] / valuation_and_quality_df['Market Cap']
                Data_for_Portfolio['Revenue'] = income_df['Revenue']
                Data_for_Portfolio['Net_Income'] = income_df['Net Income']
                Data_for_Portfolio['FCF_Margin_%'] = common_size_ratios_df['FCF Margin %']
                Data_for_Portfolio['Market Cap'] = valuation_and_quality_df['Market Cap']
                try:
                    Data_for_Portfolio['Shares Outstanding'] = valuation_and_quality_df['Shares Outstanding (EOP)']
                except:
                    pass
                Data_for_Portfolio['Intrinsic Value: Projected FCF'] = valuation_and_quality_df['Intrinsic Value: Projected FCF']
                Data_for_Portfolio['Net Margin %'] = income_df['Net Margin %']
                try:
                    Data_for_Portfolio['Operating Margin %'] = common_size_ratios_df['Operating Margin %']
                except:
                    pass
                Data_for_Portfolio['ROE %'] = common_size_ratios_df['ROE %']
                Data_for_Portfolio['ROA %'] = common_size_ratios_df['ROA %']
                Data_for_Portfolio['ROIC %'] = common_size_ratios_df['ROIC %']         
                Data_for_Portfolio['Dividend Yield %'] = valuation_ratios_df['Dividend Yield %']
                Data_for_Portfolio['YoY Rev. per Sh. Growth'] = valuation_and_quality_df['YoY Rev. per Sh. Growth']
                Data_for_Portfolio['YoY EPS Growth'] = valuation_and_quality_df['YoY EPS Growth']
                Data_for_Portfolio['Dividend Payout Ratio'] = common_size_ratios_df['Dividend Payout Ratio']   
                Data_for_Portfolio['Debt-to-Equity'] = balance_df['Debt-to-Equity']  
                try:
                    Data_for_Portfolio['market_cap'] = (valuation_and_quality_df['Shares Outstanding (EOP)'][-1] * 1000000) * yahoo_df[ticker].dropna()[-1]
                except:
                    pass           
                Data_for_Portfolio['Company'] = ticker
                Data_for_Portfolio['Date'] = date_list.tolist()              

                Data_for_Portfolio = Data_for_Portfolio.replace([np.inf, -np.inf], 0)
                Data_for_Portfolio = Data_for_Portfolio.set_index('Company')
                Data_for_Portfolio = Data_for_Portfolio[::-1].fillna(0)

                Data_for_Portfolio_TOTAL = pd.concat([Data_for_Portfolio, Data_for_Portfolio_TOTAL])
                #Data_for_Portfolio_TOTAL = Data_for_Portfolio_TOTAL[(Data_for_Portfolio_TOTAL['Date'] >= cheked_year) & (Data_for_Portfolio_TOTAL['Date'] <= cheked_year_end)]
                Data_for_Portfolio_TOTAL = Data_for_Portfolio_TOTAL[Data_for_Portfolio_TOTAL['Date'] >= cheked_year]
                    #delete "\r" to append instead of overwrite
                sys.stdout.write("\r" + str(ticker))
                sys.stdout.flush()
                time.sleep(0.3)
                
            except Exception as e:
                print(e)
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(exc_type, fname, exc_tb.tb_lineno)
                pass

    return Data_for_Portfolio_TOTAL

def yahoo_adj_close(exchange, project_id):
    #print('yahoo_adj_close')
    sql_adj_close = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Adj_Close_2`
    WHERE _Exchange in {exchange}
    """
    sql_yahoo_data_adj_close = pandas_gbq.read_gbq(sql_adj_close, project_id=project_id)
    pre_adj_close = sql_yahoo_data_adj_close.rename(columns=lambda x: x.replace('_', '-')[1:])
    pre_adj_close = pre_adj_close.T.rename(columns=sql_yahoo_data_adj_close['_Ticker'])[:-3]
    # Converting the index as date
    pre_adj_close.index = pd.to_datetime(pre_adj_close.index, format='%Y-%m-%d')
    sql_yahoo_data_adj_close = pre_adj_close[:-1]
    return sql_yahoo_data_adj_close

def yahoo_adj_close_list(company, project_id):
    #print('yahoo_adj_close')
    sql_adj_close = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Adj_Close_2`
    WHERE _Ticker in {company}
    """
    sql_yahoo_data_adj_close = pandas_gbq.read_gbq(sql_adj_close, project_id=project_id)
    pre_adj_close = sql_yahoo_data_adj_close.rename(columns=lambda x: x.replace('_', '-')[1:])
    pre_adj_close = pre_adj_close.T.rename(columns=sql_yahoo_data_adj_close['_Ticker'])[:-3]
    # Converting the index as date
    pre_adj_close.index = pd.to_datetime(pre_adj_close.index, format='%Y-%m-%d')
    sql_yahoo_data_adj_close = pre_adj_close[:-1]
    return sql_yahoo_data_adj_close

def sql_index(project_id):
    sql_index_req = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.INDEXES`
    """
    sql_index = pandas_gbq.read_gbq(sql_index_req, project_id=project_id)
    return sql_index

def sql_guru(exchange, project_id):
    if len(exchange) >= 2:
        sql_guru_req = f"""
        SELECT *
        FROM `buoyant-apogee-281013.FIN.GURU`
        WHERE exchange in {exchange}
        """
    sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
    return sql_data_guru

def sql_guru_list(company, project_id):
    sql_guru_req = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.GURU`
    WHERE company in {company}
    """
    sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
    return sql_data_guru

def sql_guru_yahoo_table(project_id):
    sql_open = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Yahoo_Exchange`
    """
    bd_data_yahoo = pandas_gbq.read_gbq(sql_open, project_id=project_id)
    return bd_data_yahoo

def sql_yahoo(ticker_list, guru_yahoo_table):
    tickers = []
    tickers_name_final = []
    for tik in ticker_list:
        #print(tik)
        if tik.split(':')[0] == 'NAS' or tik.split(':')[0] == 'NYSE':
            yah_exc = ''
        elif tik.split(':')[0] == 'AMEX':
            yah_exc = ''
        elif tik.split(':')[0] == 'HKSE':
            yah_exc = guru_yahoo_table['yahoo'].where(guru_yahoo_table['guru'] == tik.split(':')[0]).dropna().values[0]
        else:
            yah_exc = guru_yahoo_table['yahoo'].where(guru_yahoo_table['guru'] == tik.split(':')[0]).dropna().values[0]

        if tik.split(':')[0] == 'HKSE':
            tickers.append(tik.split(':')[1][1:] + yah_exc)
        else:
            tickers.append(tik.split(':')[1] + yah_exc)

    #print(tickers)
    try:
        yahoo_data = yf.download(tickers)
        # yahoo_data = yahoo_data[:limit_date]
        yahoo_data_check = yahoo_data['Adj Close']
    except:
        pass
    tickers_name_yahoo = []

    for tik_yahoo in yahoo_data_check.columns.tolist():

        if tik_yahoo.split('.')[-1] == 'HK':
            for i in range(len(ticker_list)):
                if tik_yahoo.split('.')[0] == ticker_list[i].split(':')[1][1:].split('.')[0]:
                    tickers_name_final.append(ticker_list[i])
                    tickers_name_yahoo.append(tik_yahoo)

        else:
            for i in range(len(ticker_list)):
                if tik_yahoo.split('.')[0] == ticker_list[i].split(':')[1].split('.')[0]:
                    tickers_name_final.append(ticker_list[i])
                    tickers_name_yahoo.append(tik_yahoo)

    #print('tickers_name_yahoo')
    #print(tickers_name_yahoo)
    yahoo_adj_close = pd.DataFrame()
    yahoo_adj_close = yahoo_data['Adj Close'][tickers_name_yahoo].copy().set_axis(tickers_name_final, axis=1)
    yahoo_close = yahoo_data['Close'][tickers_name_yahoo].copy().set_axis(tickers_name_final, axis=1)


    #print('yahoo_adj_close')
    #print(yahoo_adj_close)

    return yahoo_adj_close, yahoo_close

In [8]:
type_of_get_data = 'From_List'
cheked_year= '2007'
cheked_year_end = '2021'
currency = 'EUR=X'

# *****************************SQL params ***************
credentials = service_account.Credentials.from_service_account_file('gadgets_database_key.json')
project_id = 'buoyant-apogee-281013'
client = bigquery.Client(credentials=credentials, project=project_id)
table_id_guru = "buoyant-apogee-281013.FIN.GURU"
table_id_yahoo_adj_close = "buoyant-apogee-281013.FIN.Adj Close"
job_config = bigquery.QueryJobConfig()

options = { 'Hong Kong':'Hong Kong', 'China':'China', 'Japan':'Japan', 'Germany':'Germany', 'India':'India', 'Britain':'Britain', 'France':'France', \
           'Italy':'Italy', 'Brazil':'Brazil', 'Canada':'Canada', 'Russia':'Russia', 'Korea':'Korea', 'Spain':'Spain', \
           'Australia':'Australia', 'Mexico':'Mexico', 'Turkey':'Turkey', 'Netherland':'Netherland', 'Swithzerland':'Swithzerland', \
           'Poland':'Poland', 'Sweden':'Sweden', 'Thailand':'Thailand', 'Belgium':'Belgium', 'Norway':'Norway', 'Ireland':'Ireland', \
           'Israel':'Israel', 'Singapore':'Singapore', 'Malaysia':'Malaysia', 'South Africa':'South Africa', 'Philippines':'Philippines'}
Data_for_Portfolio_TOTAL_2 = pd.DataFrame()
Data_for_Portfolio_TOTAL_1 = pd.DataFrame()
def show_stats_by_tag(COUNTRY):
    
    companies = pd.read_excel('Tinkoff.xlsx', sheet_name=options[COUNTRY])['ticker Guru']
    companies = tuple(companies)
    #print('companies')
    print(companies[:10], '...')
    print(len(companies))
    company = companies
    # ===================== treads for qsl requests
    pool = ThreadPool(processes=8)

    return_sql_data_guru = pool.apply_async(sql_guru_list, (companies, project_id))
    return_sql_yahoo_data_adj_close = pool.apply_async(yahoo_adj_close_list, (companies, project_id))
    return_sql_index = pool.apply_async(sql_index, ([project_id]))
    # do some other stuff in the main process
    sql_yahoo_data_adj_close = return_sql_yahoo_data_adj_close.get()
    #sql_index = return_sql_index.get()
    sql_data_guru = return_sql_data_guru.get()

    # *****************************SQL requests ***************
    #index = sql_index['index'][0]
    tickers_yahoo = sql_yahoo_data_adj_close.columns.tolist()
    tickers_guru = sql_yahoo_data_adj_close.columns.tolist()

    data_guru = sql_data_guru[sql_data_guru['company'].isin(tickers_guru)].set_index('company')
    guru_yahoo_table = sql_guru_yahoo_table(project_id)

    yahoo_adj_close= sql_yahoo(companies, guru_yahoo_table)
    yahoo_df= pd.DataFrame(yahoo_adj_close[0])
    
    sql_guru_req = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Summary`
    WHERE company in {company}
    """

    sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
    sql_data_guru = sql_data_guru.set_index('company')

    tickers = sql_data_guru.index.tolist()
    company_data_df = []
    
    #------------------------------------------------------------------------------------------------
    Data_for_Portfolio_TOTAL = pd.DataFrame()
    Data_for_Portfolio_TOTAL = prepare_data_frame(data_guru, currency, sql_index, yahoo_df)
    global Data_for_Portfolio_TOTAL_2 
    Data_for_Portfolio_TOTAL_2  = Data_for_Portfolio_TOTAL
    global Data_for_Portfolio_TOTAL_1 
    Data_for_Portfolio_TOTAL_1  = Data_for_Portfolio_TOTAL
    print(' DONE!')
    tickers = data_guru.index.tolist()
    
    list_of_control = ['PE', 'PB', 'PS', 'PEG', 'FCF_Margin_%', 'Net Margin %', 'Operating Margin %', 'ROE %', 'ROA %', 'ROIC %', 'Dividend Yield %', 'Dividend Payout Ratio', \
                  'YoY Rev. per Sh. Growth', 'YoY EPS Growth']
    for column in list_of_control:
        for i in range(len(Data_for_Portfolio_TOTAL_2)):
            if Data_for_Portfolio_TOTAL_2[column][i] > abs(Data_for_Portfolio_TOTAL_2[column].mean())*12:
                Data_for_Portfolio_TOTAL_2[column][i] = 0
            elif Data_for_Portfolio_TOTAL_2[column][i] < 0:
                Data_for_Portfolio_TOTAL_2[column][i] = 0
    for ticker in companies:
        try:
            company_data = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['company_data']), orient='index').T
            company_data_df.append(company_data.iloc[0])
        except:
            pass          
    company_data_df = pd.DataFrame(company_data_df)
    company_data_df['rank_profitability'] = company_data_df['rank_profitability']/10
    company_data_df['p2gf_value'] = company_data_df['p2gf_value'].astype('float')
    
    Data_for_Portfolio_TOTAL_2['FCF'] = Data_for_Portfolio_TOTAL_2['Revenue']*Data_for_Portfolio_TOTAL_2['FCF_Margin_%']
    Data_for_Portfolio_TOTAL_2['Dividends payout'] = Data_for_Portfolio_TOTAL_2['Net_Income']*Data_for_Portfolio_TOTAL_2['Dividend Payout Ratio']
    Data_for_Portfolio_TOTAL_2 = Data_for_Portfolio_TOTAL_2[Data_for_Portfolio_TOTAL_2['Date']!='2021']
    Data_for_Portfolio_TOTAL_2['Date'] = Data_for_Portfolio_TOTAL_2['Date'].replace('TTM', '2021')
    portfolio_df = pd.DataFrame()
    portfolio_df['PE'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['PE'].mean()
    portfolio_df['PB'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['PB'].mean()
    portfolio_df['PS'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['PS'].mean()
    portfolio_df['PEG'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['PEG'].mean()
    portfolio_df['Revenue'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Revenue'].mean()
    portfolio_df['Net_Income'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Net_Income'].mean()
    portfolio_df['FCF'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['FCF'].mean()
    portfolio_df['Market Cap'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Market Cap'].sum()
    cagr_period = 3
    portfolio_df['Revenue CAGR'] = ''
    portfolio_df['Net_Income CAGR'] = ''
    for i in range(3,len(portfolio_df)):
        portfolio_df['Revenue CAGR'][i] = (portfolio_df['Revenue'][i]/portfolio_df['Revenue'][i-cagr_period]) ** (1/cagr_period) - 1
        portfolio_df['Net_Income CAGR'][i] = (portfolio_df['Net_Income'][i]/portfolio_df['Net_Income'][i-cagr_period]) ** (1/cagr_period) - 1
    portfolio_df['Net Margin %'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Net Margin %'].mean()
    try:
        portfolio_df['Operating Margin %'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Operating Margin %'].mean()
    except:
        pass
    portfolio_df['YoY Rev. per Sh. Growth'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['YoY Rev. per Sh. Growth'].mean()
    portfolio_df['YoY EPS Growth'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['YoY EPS Growth'].mean()
    portfolio_df['Dividend Yeld'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Dividends payout'].sum()/Data_for_Portfolio_TOTAL_2.groupby('Date')['Market Cap'].sum()
    portfolio_df['Debt-to-Equity'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Debt-to-Equity'].mean()
    portfolio_df['FCF_Margin_%'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['FCF_Margin_%'].mean()
    portfolio_df['ROE %'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['ROE %'].mean()
    portfolio_df['ROA %'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['ROA %'].mean()
    portfolio_df['ROIC %'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['ROIC %'].mean()
    portfolio_df['Dividend Yield %'] = Data_for_Portfolio_TOTAL_2.groupby('Date')['Dividend Yield %'].mean()
    portfolio_df['Revenue Growth'] = portfolio_df['Revenue'].pct_change()
    portfolio_df['Net_Income Growth'] = portfolio_df['Net_Income'].pct_change()
    portfolio_df['FCF Growth'] = portfolio_df['FCF'].pct_change()
    portfolio_df['SMA Market Cap'] = portfolio_df['Market Cap'].rolling(5).mean()
    portfolio_df['SMA PE'] = portfolio_df['PE'].rolling(5).mean()
    portfolio_df['SMA Net_Income Growth'] = portfolio_df['Net_Income Growth'].rolling(5).mean()
    #portfolio_df = portfolio_df.drop(portfolio_df.index[-1])
    #portfolio_df = portfolio_df[(portfolio_df.index >= cheked_year) & (portfolio_df.index <= cheked_year_end)]
    portfolio_df = portfolio_df[portfolio_df.index >= cheked_year]
    portfolio_df['Market Cap'][-1] =  Data_for_Portfolio_TOTAL_2.groupby('Date')['market_cap'].sum()[-1]/1000000
    


    exchange = (f'{COUNTRY}: GDP, Current Prices (National Currency)', f'Inflation, consumer prices for {COUNTRY}', \
               f'Working Age Population: Aged 15-64: All Persons for {COUNTRY}', f'Unemployment Rate: Aged 15-64: All Persons for {COUNTRY}',\
               f'Volatility of Stock Price Index for {COUNTRY}', f'{COUNTRY} Ratio of Total Market Cap over GDP',\
               f'Predicted Next 8-Year Annualized Return of {COUNTRY}', 'China: GDP, Current Prices (National Currency)', \
               'Inflation, consumer prices for China', 'Interest Rates, Discount Rate for China', 'Outstanding Domestic Private Debt Securities to GDP for China', \
               'China Ratio of Total Market Cap over GDP', 'Insider Buy / Sell Ratio - China Overall Market', \
               'Predicted Next 8-Year Annualized Return of China', 'Volatility of Stock Price Index for China')

    sql_guru_req = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Economic_Indicators`
    WHERE Indicator in {exchange}
    """

    sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
    sql_data_guru = sql_data_guru.set_index('Indicator')

    sql_data_guru['description'] # обращение к описанию отрасли
    final_df = pd.DataFrame()

    for indicator in sql_data_guru.index.tolist():
        io = json.loads(sql_data_guru.loc[indicator].loc['data'].replace('"', '~').replace("None", '"0"').replace("'", '"').replace("True", '"True"').replace("False", '"False"'))
        pr_df = pd.DataFrame(io, columns=['year', indicator]).set_index('year')
        final_df = final_df.join(pr_df, how='outer')
    final_df.index = pd.to_datetime(final_df.index)
    final_df = final_df[(final_df.index >= cheked_year) & (final_df.index <= cheked_year_end)]
    
    output_notebook()
    p = figure(title='График PE',
                 plot_height=350, plot_width=650,
                 x_axis_label='Год', y_axis_label='PE', x_axis_type="linear", toolbar_location='above')
    p.line(portfolio_df.index, portfolio_df['PE'], color='green', line_width=1)
    o = figure(title='График PB',
                 plot_height=350, plot_width=650,
                 x_axis_label='Год', y_axis_label='PB', x_axis_type="linear", toolbar_location='above')
    o.line(portfolio_df.index, portfolio_df['PB'], color='green', line_width=1)
    q = figure(title='График PS',
                 plot_height=350, plot_width=650,
                 x_axis_label='Год', y_axis_label='PS', x_axis_type="linear", toolbar_location='above')
    q.line(portfolio_df.index, portfolio_df['PS'], color='green', line_width=1)
    r = figure(title='График PEG',
                 plot_height=350, plot_width=650,
                 x_axis_label='Год', y_axis_label='PEG', x_axis_type="linear", toolbar_location='above')
    r.line(portfolio_df.index, portfolio_df['PEG'], color='green', line_width=1)
    p_panel = Panel(child=p, title='PE')
    o_panel = Panel(child=o, title='PB')
    q_panel = Panel(child=q, title='PS')
    r_panel = Panel(child=r, title='PEG')
    tabs = Tabs(tabs=[p_panel, o_panel, q_panel, r_panel])
    show(tabs)
    k = figure(title='График Доходности',
                 plot_height=400, plot_width=500,
                 x_axis_label='Год', y_axis_label='Доходность', x_axis_type="linear", toolbar_location='above')
    par = np.polyfit(portfolio_df.index.astype(float), portfolio_df['Revenue'], 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]
    y_predicted = [slope*i + intercept  for i in portfolio_df.index.astype(float)]
    k.line(portfolio_df.index, portfolio_df['Revenue'], color='green', line_width=1, legend='Revenue')
    k.line(portfolio_df.index, y_predicted, color='green', line_width=0.5, line_dash='dashed')
    # y_column2_range = 'CAGR'+ "_range"
    # k.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['Revenue CAGR'].min() * (1 - 0.1), end=portfolio_df['Revenue CAGR'].max() * (1 + 0.1))}
    # k.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
    # k.line(portfolio_df.index, portfolio_df['Revenue CAGR'], legend='Revenue CAGR', line_width=1, y_range_name=y_column2_range,  color="palegreen")
    k.legend.location = 'top_left'
    m = figure(title='График Прибыли',
                 plot_height=400, plot_width=500,
                 x_axis_label='Год', y_axis_label='Прибыль', x_axis_type="linear", toolbar_location='above')
    par1 = np.polyfit(portfolio_df.index.astype(float), portfolio_df['Net_Income'], 1, full=True)
    slope1=par1[0][0]
    intercept1=par1[0][1]
    y_predicted1 = [slope1*i + intercept1  for i in portfolio_df.index.astype(float)]
    m.line(portfolio_df.index, portfolio_df['Net_Income'], color='red', line_width=1, legend='Net_Income')
    m.line(portfolio_df.index, y_predicted1, color='red', line_width=0.5, line_dash='dashed')
    # y_column2_range = 'CAGR'+ "_range"
    # m.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['Net_Income CAGR'].min() * (1 - 0.1), end=portfolio_df['Net_Income CAGR'].max() * (1 + 0.1))}
    # m.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
    # m.line(portfolio_df.index, portfolio_df['Net_Income CAGR'], legend='Net_Income CAGR', line_width=1, y_range_name=y_column2_range,  color="lightpink")
    m.legend.location = 'top_left'
    n = figure(title='График Доходности цепной',
                 plot_height=400, plot_width=500,
                 x_axis_label='Год', y_axis_label='Доходность', x_axis_type="linear", toolbar_location='above')
    n.line(portfolio_df.index, portfolio_df['Revenue']*100/portfolio_df['Revenue'][0], color='green', line_width=1, legend='Revenue')
    n.line(portfolio_df.index, portfolio_df['Net_Income']*100/portfolio_df['Net_Income'][0], color='red', line_width=1, legend='Net_Income')
    n.line(portfolio_df.index, portfolio_df['FCF']*100/portfolio_df['FCF'][0], color='blue', line_width=1, legend='FCF')
    n.line(portfolio_df.index, portfolio_df['Market Cap']*100/portfolio_df['Market Cap'][0], color='orange', line_width=1, legend='Market Cap')
    n.legend.location = 'top_left'
    l = figure(title='График Роста',
                 plot_height=400, plot_width=500,
                 x_axis_label='Год', y_axis_label='Рост', x_axis_type="linear", toolbar_location='above')
    l.line(portfolio_df.index, portfolio_df['YoY Rev. per Sh. Growth'], color='green', line_width=1, legend='YoY Rev. per Sh. Growth')
    l.line(portfolio_df.index, portfolio_df['YoY EPS Growth'], color='red', line_width=1, legend='YoY EPS Growth')
    l.legend.location = 'top_left'
    show(row(k,m))
    show(row(n,l))
    s = figure(title='График Маржинальностей',
                 plot_height=400, plot_width=650,
                 x_axis_label='Год', y_axis_label='Margin %', x_axis_type="linear", toolbar_location='above')
    s.line(portfolio_df.index, portfolio_df['Net Margin %'], color='green', line_width=1, legend='Net Margin %')
    s.line(portfolio_df.index, portfolio_df['Operating Margin %'], color='blue', line_width=1, legend='Operating Margin %')
    s.line(portfolio_df.index, portfolio_df['FCF_Margin_%'], color='red', line_width=1, legend='FCF_Margin_%')
    s.legend.location = 'bottom_left'
    t = figure(title='График Эффективностей',
                 plot_height=400, plot_width=650,
                 x_axis_label='Год', y_axis_label='%', x_axis_type="linear", toolbar_location='above')
    t.line(portfolio_df.index, portfolio_df['ROE %'], color='green', line_width=1, legend='ROE %')
    t.line(portfolio_df.index, portfolio_df['ROA %'], color='blue', line_width=1, legend='ROA %')
    t.line(portfolio_df.index, portfolio_df['ROIC %'], color='red', line_width=1, legend='ROIC %')
    t.legend.location = 'bottom_left'
    u = figure(title='График объема Дивидендов',
                 plot_height=400, plot_width=650,
                 x_axis_label='Год', y_axis_label='%', toolbar_location='above')
    u.line(portfolio_df.index, portfolio_df['Dividend Yeld'], color='green', line_width=1, legend='Dividend Yeld')
    u.legend.location = 'bottom_left'
    v = figure(title='График Debt-to-Equity',
                 plot_height=400, plot_width=650,
                 x_axis_label='Год', y_axis_label='%', toolbar_location='above')
    v.line(portfolio_df.index, portfolio_df['Debt-to-Equity'], color='green', line_width=1, legend='Debt-to-Equity')
    v.legend.location = 'bottom_left'
    s_panel = Panel(child=s, title='Маржинальности')
    t_panel = Panel(child=t, title='Эффективность')
    u_panel = Panel(child=u, title='Дивиденды')
    v_panel = Panel(child=v, title='Debt-to-Equity')
    tabs2 = Tabs(tabs=[s_panel, t_panel, u_panel, v_panel])
    show(tabs2)
    w = figure(title='График Сравнения',
             plot_height=400, plot_width=900,
             x_axis_label='Год', x_axis_type="linear")
    #w.line(portfolio_df.index, portfolio_df['SMA Market Cap'], color='orange', line_width=1, legend='SMA Market Cap')
    w.line(portfolio_df.index, portfolio_df['Market Cap'], color='orange', line_width=1, legend='Market Cap')
    #w.line(portfolio_df.index, portfolio_df['Net_Income Growth']*1000, legend='Net_Income', line_width=1, color="blue")
    # SECOND AXIS
    y_column2_range = 'PE'+ "_range"
    w.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['PE'].min() * (1 - 0.1), end=portfolio_df['PE'].max() * (1 + 0.1))}
    w.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
    w.line(portfolio_df.index, portfolio_df['SMA PE'], legend='SMA PE', line_width=1, y_range_name=y_column2_range,  color="green")
    w.line(portfolio_df.index, portfolio_df['PE'], legend='PE', line_width=1, y_range_name=y_column2_range, color="blue")
    #w.line(portfolio_df.index, portfolio_df['SMA Net_Income Growth']/portfolio_df['SMA Net_Income Growth'][0], legend='SMA Net_Income Growth', line_width=1, color="blue")
    w.legend.location = 'top_left'
    n = figure(title='График Ростов',
                 plot_height=300, plot_width=900,
                 x_axis_label='Год', y_axis_label='Net_Income', x_axis_type="linear")
    n.line(portfolio_df.index, portfolio_df['Net_Income'], color='red', line_width=1, legend='Net_Income')
    # SECOND AXIS
    # y_column2_range = 'SMA Net_Income Growth'+ "_range"
    # n.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['SMA Net_Income Growth'].min() * (1 - 0.1), end=portfolio_df['SMA Net_Income Growth'].max() * (1 + 0.1))}
    # n.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
    # n.line(portfolio_df.index, portfolio_df['SMA Net_Income Growth'], color='red', y_range_name=y_column2_range, line_width=1, legend='SMA Net_Income Growth')
    # n.legend.location = 'top_left'
    show(w)
    show(n)
    print(company_data_df[['p2gf_value', 'rank_financial_strength', 'rank_profitability', 'rank_value']].mean().round(2))
        
    try:
        gdp = final_df[f'{COUNTRY}: GDP, Current Prices (National Currency)'].dropna()
        gdp_ch = final_df['China: GDP, Current Prices (National Currency)'].dropna()
        x = figure(title=f'{COUNTRY}: GDP, Current Prices (National Currency)', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        x.line(gdp.index, gdp, color='green', line_width=1, legend='GDP')
        x.line(gdp.index, gdp_ch, color='red', line_width=1, legend='GDP_China')
        x.legend.location = 'top_left'
        inflation = final_df[f'Inflation, consumer prices for {COUNTRY}'].dropna()
        inflation_ch = final_df['Inflation, consumer prices for China'].dropna()
        y = figure(title='Inflation, consumer prices', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        y.line(inflation.index, inflation, color='green', line_width=1, legend=f'Inflation, consumer prices')
        y.line(inflation.index, inflation_ch, color='red', line_width=1, legend=f'Inflation, consumer prices')
        y.legend.location = 'top_left'
        show(row(x,y))
    except:
        pass
    try:
        interest = final_df['Interest Rates, Discount Rate for China'].dropna()
        z = figure(title='Interest Rates, Discount Rate for China', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        z.line(interest.index, interest, color='green', line_width=1, legend='Interest Rates, Discount Rate')
        z.legend.location = 'top_left'
        show(row(z))
    except:
        pass
    try:
        debt = final_df['Outstanding Domestic Private Debt Securities to GDP for China'].dropna()
        a = figure(title='Outstanding Domestic Private Debt Securities to GDP for China', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        a.line(debt.index, debt, color='green', line_width=1, legend='Outstanding Domestic Private Debt Securities to GDP')
        a.legend.location = 'top_left'
        show(row(a))
    except:
        pass
    try:
        market = final_df['China Ratio of Total Market Cap over GDP'].dropna()
        market_cap = final_df[f'{COUNTRY} Ratio of Total Market Cap over GDP'].dropna()
        e = figure(title='China Ratio of Total Market Cap over GDP', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        e.line(market.index, market, color='red', line_width=1, legend='China Ratio of Total Market Cap over GDP')
        e.line(market.index, market_cap, color='green', line_width=1, legend='Ratio of Total Market Cap over GDP')
        show(row(e))
    except:
        pass
    try:
        insider = final_df['Insider Buy / Sell Ratio - China Overall Market'].dropna()
        g = figure(title='Insider Buy / Sell Ratio - China Overall Market', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        g.line(insider.index, insider, color='red', line_width=1, legend='Insider Buy / Sell Ratio - China Overall Market')
        g.legend.location = 'top_left'
        show(row(g))
    except:
        pass
    try:
        returns = final_df['Predicted Next 8-Year Annualized Return of China'].dropna()
        b = figure(title='Predicted Next 8-Year Annualized Return of China', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        b.line(returns.index, returns, color='red', line_width=1, legend='Predicted Next 8-Year Annualized Return of China')
        b.legend.location = 'top_left'
        show(row(b))
    except:
        pass
    try:
        vol = final_df['Volatility of Stock Price Index for China'].dropna()
        c = figure(title='Volatility of Stock Price Index for China', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
        c.line(vol.index, vol, color='red', line_width=1, legend='Volatility of Stock Price Index for China')
        c.legend.location = 'top_left'
        show(row(c))
    except:
        pass
    return

### Выбрать страну

In [9]:
stats = interact(show_stats_by_tag,
                COUNTRY=widgets.Dropdown(options = ['Hong Kong', 'China', 'Japan', 'Germany', 'India', 'Britain', 'France', 'Italy', 'Brazil', 'Canada', 'Russia', 'Korea', 'Spain', \
                                                'Australia', 'Mexico',  'Turkey', 'Netherland', 'Swithzerland', 'Poland', 'Sweden', 'Thailand',  'Belgium', 'Norway', \
                                                'Ireland', 'Israel', 'Singapore', 'Malaysia',  'South Africa', 'Philippines'], value="Hong Kong"))

interactive(children=(Dropdown(description='COUNTRY', options=('Hong Kong', 'China', 'Japan', 'Germany', 'Indi…